# Data Collection
This noteboook is responsible for (finish later)

## Imports

In [3]:
#SQL connection and queries
import MySQLdb as mdb
#Connection failure exiting
import sys
#Getting current time (after data is scraped) for stream table
from time import time, sleep
#Scraping
import urllib2
#Web automation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## File data
Below the database credentials are read in. These are used for logging into the database, creating the necessary tables (if not already created), and storing data.

In [4]:
db_user = ""
db_pass = ""
db_name = ""
db_host = "localhost"
with open("database_credentials.txt") as f:
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_name = f.readline().strip()

## Database

### Table Creation
The below functions create the necessary tables for data storage if they do not already exist.

In [6]:
con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)

with con:
    cur = con.cursor()

    #Twitch game/creative/irl categories
    sql = """
    CREATE TABLE IF NOT EXISTS tweets (
        id INT PRIMARY KEY AUTO_INCREMENT,
        message VARCHAR(255) NOT NULL UNIQUE
    )
    """
    cur.execute(sql)
#create_tables()

### URL strings

In [7]:
#String for vieweing all current streams for a specified category
twitter_string = "https://twitter.com/realdonaldtrump"

## Data collection

### Functional output separation
The below code just prints out visual separation when the loop is performing different tasks. Separation is used to divide output by category, scraping, and writing.

In [21]:
def output_separation(phrase, sep_char, border_height, border_width):
    """
    This funciton provides the following functionality:
        output_separation("hi", "|", 3, 10) yields
        ||||||||||
        ||||||||||
        ||||||||||
        ||||hi||||
        ||||||||||
        ||||||||||
        ||||||||||
    Note that sep_char must be a single character.
    """
    for i in range(border_height):
        print(sep_char*border_width)
    print(phrase.center(border_width, sep_char))
    for i in range(border_height):
        print(sep_char*border_width)

### Scraping
(finish later)

In [19]:
#Spin up browser
driver = webdriver.Chrome()
driver.get(twitter_string)

#tweets = driver.find_elements_by_class_name("js-tweet-text");
#tweets = map(lambda e : e.get_attribute("innerHTML"), tweets)
shit = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
print(shit)


      <script id="swift_loading_indicator" nonce="">
        document.body.className=document.body.className+" "+document.body.getAttribute("data-fouc-class-names");
      </script>

    <a href="#timeline" class="u-hiddenVisually focusable">Skip to content</a>

    
    
    
    
    
    
    
    
    
    <div id="doc" data-at-shortcutkeys="{&quot;Enter&quot;:&quot;Open Tweet details&quot;,&quot;o&quot;:&quot;Expand photo&quot;,&quot;/&quot;:&quot;Search&quot;,&quot;?&quot;:&quot;This menu&quot;,&quot;j&quot;:&quot;Next Tweet&quot;,&quot;k&quot;:&quot;Previous Tweet&quot;,&quot;Space&quot;:&quot;Page down&quot;,&quot;.&quot;:&quot;Load new Tweets&quot;,&quot;gu&quot;:&quot;Go to user\u2026&quot;}" class="route-profile">
        <div class="topbar js-topbar">
    


    <div class="global-nav global-nav--newLoggedOut" data-section-term="top_nav">
      <div class="global-nav-inner">
        <div class="container">

          
<ul class="nav js-global-actions" role="navigation" id=

In [22]:
first_time = time()

In [23]:
#Spin up browser
driver = webdriver.Chrome()

#Number of times to scroll down a page at a time
scrolls = 5

#Time to sleep between scrolls
sleeptime = 0.5

#Used for output formatting
out_width = 72

#Iterate over each category
for c in category_map:
    output_separation(c, " ", 8, out_width)
    start = time()
    #Load up page
    driver.get(category_map[c]["url"])
    
    #Streams for storage
    streams = []
    stream_map = {}
    
    try:
        #Wait for page to load at least 1 stream
        element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "stream-thumbnail")))
        
        #Loop to get more streams
        output_separation("COLLECTING STREAM EVENTS", "|", 4, out_width)
        prev_stream_count = -1
        while True:            
            #Assign global scrollable element for access through other scripts
            scroll_script = """
            window.s = document.getElementsByClassName("root-scrollable__wrapper")[0].parentElement.parentElement;
            """
            driver.execute_script(scroll_script)
            
            #Scroll
            scroll_script = """
            window.s.scrollTo(0, window.s.scrollHeight);
            """
            output_separation("SCROLLING FOR {}s".format(scrolls * sleeptime), "#", 0, out_width)
            for i in range(scrolls):
                driver.execute_script(scroll_script)
                sleep(sleeptime)
                
            #Find streams
            streams = driver.find_elements_by_class_name("live-channel-card__videos")
            #Find text in parent containing concurrent viewers
            streams = map(lambda e: e.find_element_by_xpath("..").text, streams)
            
            #If last stream obtained has 0 viewers, break
            if prev_stream_count == len(streams):
                break
            prev_stream_count = len(streams)
    finally:
        output_separation("RETRIEVAL AND UNWRAPPING: {}s".format(time() - start), "~", 1, out_width)
    
    #Iterate over streams
    output_separation("SCRAPING STREAM DATA", "-", 4, out_width)
    for s in streams:
        #Add kvp mapping for streamer
        streamer = s.split(" ")[-1]
        #Foreign players with non-english characters have "(name_here)"
        if "(" in streamer:
            streamer = streamer[1:-1]
        
        #Load up page
        print("{}".ljust(40 - len(streamer), " ").format(streamer)),
        driver.get(stream_string.format(streamer))
        try:
            #Wait for page to load total viewer count
            #element = WebDriverWait(driver, 5).until(
            #        EC.text_to_be_present_in_element((By.CLASS_NAME, "tw-stat__value")))
            sleep(1)
            
            #Record stats
            #Followers
            top_stats = driver.find_elements_by_class_name("channel-header__item-count")
            if len(top_stats) == 0:
                #Channel lookup error or banned (banned unlikely)
                print("[BANNED->SKIPPED]")
                continue
            followers = twitch_numberfy(
                top_stats[1].find_element_by_tag_name("span").get_attribute("innerHTML"))
            #Current viewers and total views
            bottom_stats = driver.find_elements_by_class_name("tw-stat__value")
            if len(bottom_stats) == 0:
                #Hosting, so skip
                print("[HOSTING->SKIPPED]")
                continue
            elif len(bottom_stats) == 1:
                #Offline, so skip
                print("[OFFLINE->SKIPPED]")
                continue
            current_viewers = twitch_numberfy(bottom_stats[0].get_attribute("innerHTML"))
            total_views = twitch_numberfy(bottom_stats[1].get_attribute("innerHTML"))
            
            #Store in dictionary
            stream_map[streamer] = {
                "followers": followers,
                "current_viewers": current_viewers,
                "total_views": total_views
            }
            print("[DONE]")
            print("\tfollowers: {}\n\tcurrent_viewers: {}\n\ttotal_views: {}".format(
                followers,
                current_viewers,
                total_views
            ))
        except (RuntimeError, TypeError, NameError):
            print("[ERROR->SKIPPED]")
            
    #Write streams and streamers to database
    output_separation("STORING EVENTS AND STREAMERS", "*", 4, out_width)
    try:
        #Connect
        con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
        #Set autocommit (otherwise inserts won't be propogated to database)
        con.autocommit(True)
        cur = con.cursor(mdb.cursors.DictCursor)
        
        #Iterate over streams for this category
        category_id = category_map[c]["primary_key"]
        for k,v in stream_map.iteritems():
            #k is streamer (twitch name)
            #v is dictionary holding followers, current_viewers, and total_views
            
            #Get streamer ids. If the streamer is undiscovered/new, create
            #a database entry first
            sql = """
            SELECT * FROM streamers WHERE streamer='{}'
            """.format(k)
            result = cur.execute(sql)
            #print("Result: {}".format(result))

            streamer_id = -1
            #If 0 < result, then streamer already has an entry, so get their primary key
            if 0 < result:
                #Only 1 can exist, the varchar is unique
                streamer_id = cur.fetchall()[0]["id"]
            #Otherwise, this is a new streamer
            else:
                #Insert new entry
                sql = """
                INSERT INTO streamers (streamer) VALUES ('{}')
                """.format(k)
                cur.execute(sql)
                #Get primary key of last row inserted into ANY table
                streamer_id = cur.lastrowid
            
            #Streamer information has been obtained (and added if necessary).
            #Create stream event entry
            sql = """
            INSERT INTO streams
                (current_viewers, followers, total_views, streamer_id, category_id)
                VALUES
                ({}, {}, {}, {}, {})
            """.format(
                v["current_viewers"],
                v["followers"],
                v["total_views"],
                streamer_id,
                category_id)
            cur.execute(sql)
            
            #Output storage
            print("{}".ljust(40 - len(k), " ").format(k)),
            print("[OKAY]")
    finally:
        if con:
            con.close()
#Close browser
driver.close()

                                                                        
                                                                        
                                                                        
                                                                        
                                                                        
                                                                        
                                                                        
                                                                        
                           pokémon red/blue                            
                                                                        
                                                                        
                                                                        
                                                                        
                                                    

Zomby_Goast                            [DONE]
	followers: 17
	current_viewers: 2
	total_views: 262
TheOnlyXanzu                           [DONE]
	followers: 57
	current_viewers: 2
	total_views: 2013
Kei_SiLiC                              [DONE]
	followers: 14
	current_viewers: 0
	total_views: 229
************************************************************************
************************************************************************
************************************************************************
************************************************************************
**********************STORING EVENTS AND STREAMERS**********************
************************************************************************
************************************************************************
************************************************************************
************************************************************************


IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (`proj`.`streams`, CONSTRAINT `streams_ibfk_2` FOREIGN KEY (`category_id`) REFERENCES `categories` (`id`) ON DELETE CASCADE)')

In [ ]:
print("total time: {}s".format(time() - first_time))